In [3]:
from sklearn.linear_model import SGDClassifier
import numpy as np
import collections
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor
import geometry
import itertools
from collections import defaultdict
import collections
import target
import controler
import utils
from functools import reduce
import linear_regression
from multiprocessing import Pool
from functools import partial


class SyntheticCase:

    def __init__(self, LossMatrix, FeedbackMatrix, horizon ):
 
        self.LossMatrix = LossMatrix 
        self.FeedbackMatrix = FeedbackMatrix 
        self.horizon = horizon
        self.n_actions = len(self.LossMatrix)

    def parameters_Bianchi(self, C):
        # [Bianchi et al. 2006 "Regret minimization under partial monitoring"]
        eta = 1 / C * pow( np.log( self.n_actions ) / ( self.n_actions * self.horizon ) , 2./3.) 
        gamma = C * pow( ( np.log( self.n_actions ) * self.n_actions **2) / self.horizon , 1./3.)
        return eta, gamma 

    def parameters_Piccolboni(self, ):
        ## [Piccolboni Schindelhauer "Discrete Prediction Games with Arbitrary Feedback and Loss" 2000]
        ## fixed-known-horizon settings
        eta = pow( np.log(self.n_actions), 1./2.) / pow(self.horizon, 1./2.)
        gamma = np.fmin(1., pow(self.n_actions, 1./2.) * pow( np.log(self.n_actions),1./4.) / pow(self.horizon, 1./4.))
        return eta, gamma
        
    def feedexp3(self, method, job_id):
        np.random.seed(job_id)

        self.LinkMatrix = np.linalg.lstsq(self.FeedbackMatrix, self.LossMatrix,rcond=None )[0]
        k_star = max( [1, np.fabs(self.LinkMatrix).max() ] )
        C = pow( k_star * np.sqrt(np.exp(1.)-2.), 2./3.)

        w = np.ones(self.n_actions)
        cumRegret = []
        cumAllLosses, cumSufferedLoss = 0 , 0 

        if method == 'Bianchi' :
            eta, gamma = self.parameters_Bianchi(C) 
        else:
            eta, gamma = self.parameters_Piccolboni()

        for t in range(self.horizon):

            pbt = [ (1-gamma) * w[i] / sum(w)  + gamma/self.n_actions for i in range(self.n_actions)]
            action = np.random.choice([0,1], p=pbt )

            outcome = np.random.choice([0,1], p=[0.9, 0.1] )
            feedback = self.FeedbackMatrix[action, outcome]

            for i in range(self.n_actions):
                l_i = self.LinkMatrix[i,action] * feedback / pbt[action]
                w[i] = w[i] * np.exp(-eta * l_i)

            # policy suffers loss and regret
            cumAllLosses += self.LossMatrix[...,outcome]
            cumSufferedLoss += self.LossMatrix[action,outcome]
            cumRegret.append(  cumSufferedLoss - min(cumAllLosses) )

        return np.array(cumRegret)

    def f(self, alpha, t):
        return (alpha ** 1/3) * (t**2/3) * ( np.log(t)**1/3 )

    def W_k(self, L, H, pair,mathcal_N_plus):
        v_ij = geometry.observer_vector(L, H, pair[0], pair[1], mathcal_N_plus)[1]

        res =  max( [ np.fabs( v_ij[k]).max() for k in mathcal_N_plus ]  )
        # print(res)
        return res

    # def get_confidence(x_t, t):
    #     d = 1 # what is the value of d?
    #     res = d * np.sqrt( d * np.log(t) + 2 * np.log( 1/ d_t)  )  * np.norm( x_t )
    #     return 

    def cpb_vanilla(self,):

        cumRegret = []
        cumAllLosses, cumSufferedLoss = 0 , 0 
        
        alpha = 1.1 # alpha > 1
        L = np.array([ [0,1],[1,0] ])
        H = np.array([ [1., 1.], [1., 0.] ] )
        N_bar = [0,1]
        N = len(L)
        n = np.zeros(N)
        v = np.zeros(N)
        mathcal_P = [ a for a in N_bar if geometry.isParetoOptimal(1, L)] # set of pareto optimal actions
        mathcal_N = [ pair for pair in list( itertools.combinations([0,1], 2) ) if geometry.areNeighbours(pair[0], pair[1], L) ] #set of unordered neighboring actions

        #mathcal_N_plus = [ geometry.get_neighborhood_action_set(pair, N_bar, L) for pair in mathcal_N] 

        mathcal_N_plus = [ geometry.get_neighborhood_action_set(pair, N_bar, L) for pair in mathcal_N]  #neighborhood action set of pair 
        v_ij = [ geometry.observer_vector(L, H, pair[0], pair[1], mathcal_N_plus) for pair in mathcal_N]
        print(v_ij)

        outcomes = np.random.choice([0,1], p=[0.9, 0.1],size= self.horizon)

        for t in range(self.horizon):

            if t < N:  # initialisation
                I  = t
                J = outcomes[t]
                feedback = H[ I ][ J ]
                n[I] += 1
                v[I] += feedback

                cumAllLosses += L[...,J]
                cumSufferedLoss += L[I,J]
                cumRegret.append(  cumSufferedLoss - min(cumAllLosses) )

            else: 
                break
                
        for t in range(self.horizon):
            J = outcomes[t]
            half_space = collections.defaultdict(dict)

            if t >= N:

                for pair in mathcal_N:
    
                    d_ij = sum( [  v_ij[1][k].T * v[k]/n[k]   for k in mathcal_N_plus ] )
                    c_ij = sum( [  np.fabs( v_ij[1][k]).max()  * np.sqrt(alpha * np.log(t) / n[k] )    for k in mathcal_N_plus ] )

                    if abs( d_ij ) >= c_ij:
                        half_space[ pair[0] ][ pair[1] ] = np.sign(d_ij)
                    else:
                        half_space[ pair[0] ][ pair[1] ] = 0

                mathcal_P, mathcal_N = geometry.get_polytope(half_space, L, mathcal_P, mathcal_N)

                Q = [  ]
                for k in N_bar:
                    for pair in mathcal_N:
                        mathcal_N_plus =  geometry.Neighbourhood(pair[0], pair[1], L)
                        if k in mathcal_N_plus:
                            Q.append(k)

                
                values = [ self.W_k(L, H, pair,mathcal_N_plus)/n[k] for k in Q ]
                print('values', values)
                
                I = np.argmax(values)
                feedback = H[ I ][ J ]
                n[I] += 1
                v[I] += feedback

                cumAllLosses += L[...,J]
                cumSufferedLoss += L[I,J]
                cumRegret.append(  cumSufferedLoss - min(cumAllLosses) )

        return np.array(cumRegret)




def eval_policy_parallel(nbCores, nbReps, horizon, method):
    LossMatrix, FeedbackMatrix = np.array([ [1., 0.], [0., 1.] ]) , np.array([ [1., 1.], [1., 0.] ] )
    print("nbCores:", nbCores, "nbReps:", nbReps, "Horizon:", horizon)
    pool = Pool(processes = nbCores) 
    task = SyntheticCase(LossMatrix, FeedbackMatrix, horizon) 
    return np.asarray(  pool.map( partial(task.feedexp3,method), range(nbReps) ) ) 



n_cores = 1
horizon = 1000
n_folds = 5

LossMatrix, FeedbackMatrix = np.array([ [1., 0.], [0., 1.] ]) , np.array([ [1., 1.], [1., 0.] ] )
task = SyntheticCase(LossMatrix, FeedbackMatrix, horizon) 
result = task.cpb_vanilla()
plt.plot( result, label = 'CPB-vanilla' )

# result = eval_policy_parallel(n_cores, n_folds, horizon,'Bianchi' )
# mean = np.mean(  result,0)
# plt.plot( mean, label = 'Bianchi' )
# plt.fill_between( range(horizon), mean -  np.std(result, axis=0) / np.sqrt(n_folds), mean +  np.std(result, axis=0) / np.sqrt(n_reps), alpha=0.2, color = 'blue') 

# result = eval_policy_parallel(n_cores, n_folds, horizon,'Picolboni' )
# mean = np.mean(  result,0)
# plt.plot( mean, label = 'Picolboni' )
# plt.fill_between( range(horizon), mean -  np.std(result, axis=0) / np.sqrt(n_folds), mean +  np.std(result, axis=0) / np.sqrt(n_reps), alpha=0.5, color = 'orange') 

# plt.xlabel('Iteration')
# plt.ylabel('Cumulative Regret')
# plt.legend()
    

True
True
[[array([-0.,  1.])]]


/home/mheuillet/Desktop/attack-detection/geometry.py:40: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x, res, rank, s = np.linalg.lstsq(A.T, Lij)


IndexError: list index out of range

In [19]:
import numpy as np
import geometry
import itertools 
L = np.array([ [0,1],[1,0] ])
H = np.array([ ['no_feedback','no_feedback'], [0,1] ], dtype=object)
N_bar = [0,1]
mathcal_M = [0,1, 'no_feedback']
N = len(L)
n = np.zeros(N)
mathcal_P = [ a for a in N_bar if geometry.isParetoOptimal(1, L)] # set of pareto optimal actions
mathcal_N = [ pair for pair in list( itertools.combinations([0,1], 2) ) if geometry.areNeighbours(pair[0], pair[1], L) ] #set of unordered neighboring actions
print(mathcal_N)
mathcal_N_plus = collections.defaultdict(dict)
for pair in mathcal_N:
        mathcal_N_plus[ pair[0] ][ pair[1] ] = geometry.get_neighborhood_action_set(pair, N_bar, L)

observer_set = collections.defaultdict(dict)
for pair in mathcal_N : 
        if geometry.ObservablePair(pair[0], pair[1], L, [geometry.signal_vecs(i, H) for i in geometry.Neighbourhood(0, 1, L)]):
                observer_set [ pair[0] ][ pair[1] ] =   mathcal_N_plus[ pair[0] ][ pair[1] ] 
        else:
                observer_set [ pair[0] ][ pair[1] ] = None
                print('Observer set -- not implemented')

observer_vector = collections.defaultdict(dict)
for pair in mathcal_N :
        observer_set[ pair[0] ][ pair[1] ]  

# Lij = L[0,...] - L[1,...]

# observer_vector = []
# for pair in mathcal_N:

# observer_set = [   for pair in mathcal_N ]

# A = np.vstack( A )
# print(A)
        
        # np.vstack( global_signal(H) )
        # v_ij = geometry.observer_vector(L, H, pair[0], pair[1], mathcal_N_plus)



# def observer_vector(L, H, i, j, mathcal_K_plus):
#     A = np.vstack( global_signal(H) )
#     Lij = L[i,...] - L[j,...]
#     # print('Lij', Lij)
#     # print('globalsignal',global_signal(H))
#     x, res, rank, s = np.linalg.lstsq(A.T, Lij) 
#     lenght = [ len( np.unique(H[k]) ) for k in mathcal_K_plus]
#     x = iter( np.round(x) )
#     return [ np.array( list(islice( x, i)) ) for i in lenght] 

# print( geometry.global_signal(H) )

# print( geometry.signal_vecs(0, H) )
# print( np.array(geometry.signal_vecs(1, H) ) )
# product =  geometry.signal_vecs(0, H)[0] *  geometry.signal_vecs(1, H) 
# pinv = np.linalg.pinv( product )

# print(product)
# print(pinv)
# print( Lij )
# print( pinv * Lij )
# print()
# print( geometry.observer_vector(L, H, 0, 1, mathcal_N_plus[0]) )

[(0, 1)]
True
True


In [45]:
geometry.global_signal(H)

[[array([1., 1.])], [array([1., 0.]), array([0., 1.])]]

In [62]:
from itertools import islice

def get_observer_vector(pair):
    
    Lij = L[pair[0],...] - L[pair[1],...]
    S_vectors = [ geometry.signal_vecs(k, H) for k in observer_set[ pair[0] ][ pair[1] ] ]
    print(S_vectors)
    stacked_S =  np.linalg.pinv(  np.vstack( S_vectors ).T )

    resultat = stacked_S * Lij 
    v_ij = resultat.T[0]
    lenght = [ len(k)  for k in S_vectors]
    v_ij = iter( v_ij )
    return [ np.array( list( islice( v_ij, i)) ) for i in lenght] 


[-1  1]
[[ 0.33333333  0.33333333]
 [ 0.66666667 -0.33333333]
 [-0.33333333  0.66666667]]
[[-0.33333333 -0.66666667  0.33333333]
 [ 0.33333333 -0.33333333  0.66666667]]
[-0.33333333 -0.66666667  0.33333333]
